In [3]:
import pandas as pd
from meetup import get_meetup_events
from luma import get_luma_group_events
from common import upload_to_gsheets, get_gsheet_df, create_event_markdown, create_discord_list
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import os
import json
import asyncio
from pydantic import BaseModel
from crawl4ai import AsyncWebCrawler, CrawlerRunConfig
from crawl4ai.extraction_strategy import LLMExtractionStrategy
# from llm_scrape import extract_events_with_llm

# TODO: Get 1871 Events
# TODO: Get lu.ma users Events
# TODO: Fix luma
# TODO: Get Eventbrite Events
# TODO: Figure out the list of others that need to be scraped
# TODO: Figure out a way to make this format nicely for pasting into slack/discord
# TODO: Make a website/calendar
# TODO: Filter non-Chicago events from meetup groups (e.g. https://www.meetup.com/pydatachi/events/303000483/)
# TODO: Split slack messages into multiple markdowns to make life easier
# TODO: Throw this into ChatGPT to simplify the list?
# TODO: Add Organization to Markdown

In [ ]:
from llm_scrape import scrape_events

event_urls = [
    'https://calendar.time.ly/s9lik1ia/event/77881281/20250319110000?r=https:%2F%2Fmn.gov%2Flaunchmn%2Fcalendar%2F;event%3D77881281;instance%3D20250319110000&popup=1',
    # other URLs...
]

# This will work in a Jupyter notebook
event_data = scrape_events(event_urls)
print(f"Scraped {len(event_data)} events")

## MSP Organizations

In [3]:
organization_df = get_gsheet_df("MSP Orgs")

In [4]:
meetup_groups = organization_df['Meetup'].tolist()
meetup_groups = list(set([x for x in meetup_groups if len(x) > 0]))

In [ ]:
meetup_df = get_meetup_events(meetup_groups, include_virtual=True, sleep_time=2, verbose=True, debug=True)

In [4]:

event_urls = [
    'https://calendar.time.ly/s9lik1ia/event/77881281/20250319110000?r=https:%2F%2Fmn.gov%2Flaunchmn%2Fcalendar%2F;event%3D77881281;instance%3D20250319110000&popup=1',
    'https://carlsonschool.umn.edu/faculty-research/gary-s-holmes-center-entrepreneurship/blog/foundersday2023',
    'https://events.humanitix.com/angel-fest-2025/tickets',
    'https://hclib.bibliocommons.com/events/6759fe81d6b435360093197f',
    'https://hclib.bibliocommons.com/events/6759feaed6b4353600931a03',
    'https://hclib.bibliocommons.com/events/6759ff2717cee0280055f62b',
    'https://hclib.bibliocommons.com/events/6759ff68da3a5e3d00ca4d78',
    'https://hclib.bibliocommons.com/events/67807909b4127941005f7a6c',
    'https://hclib.bibliocommons.com/events/67807ab05ca5ec68c1c0ec4b']


Task exception was never retrieved
future: <Task finished name='Task-2' coro=<Connection.run() done, defined at C:\Users\drisc\.conda\envs\chicago-engineering-and-tech-communities\Lib\site-packages\playwright\_impl\_connection.py:272> exception=NotImplementedError()>
Traceback (most recent call last):
  File "C:\Users\drisc\.conda\envs\chicago-engineering-and-tech-communities\Lib\asyncio\tasks.py", line 314, in __step_run_and_handle_result
    result = coro.send(None)
             ^^^^^^^^^^^^^^^
  File "C:\Users\drisc\.conda\envs\chicago-engineering-and-tech-communities\Lib\site-packages\playwright\_impl\_connection.py", line 279, in run
    await self._transport.connect()
  File "C:\Users\drisc\.conda\envs\chicago-engineering-and-tech-communities\Lib\site-packages\playwright\_impl\_transport.py", line 133, in connect
    raise exc
  File "C:\Users\drisc\.conda\envs\chicago-engineering-and-tech-communities\Lib\site-packages\playwright\_impl\_transport.py", line 120, in connect
    sel

NotImplementedError: 

In [ ]:
# load event_data.json
with open('event_data.json', 'r') as f:
    event_data = json.load(f)

# convert to DataFrame
event_df = pd.DataFrame(event_data)

# in the details column there is a list, replace it with the first entry
event_df['details'] = event_df['details'].apply(lambda x: x[0])

# in the details column nwo there is a dictionary, with the fields event_title, event_start_time, event_end_time, event_description, event_organizer, event_location, make these into columns
event_df = pd.concat([event_df.drop(['details'], axis=1), event_df['details'].apply(pd.Series)], axis=1)

event_df


In [ ]:
combined_df = pd.concat([meetup_df], ignore_index=True)

In [ ]:
upload_to_gsheets(combined_df, "MSP Events", ['eventURL'], verbose=True)

In [ ]:
events_df = get_gsheet_df("MSP Events")
# filter out where the eventVenueName is Online
events_df = events_df[events_df['eventVenueName'] != "Online"]
create_event_markdown(events_df, "MSP Events")

events_df = get_gsheet_df("MSP Events")
events_df = events_df[events_df['eventVenueName'] != "Online"]
create_discord_list(events_df, "MSP Events")

In [10]:
events_df = get_gsheet_df("MSP Events")

# filter out where Submitted is True
events_df = events_df[events_df['Submitted'] != "TRUE"]


def clean_dataframe_text(df):
    # Replace newlines and quotes in string columns
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].astype(str).str.replace('"', "'")
        df[col] = df[col].str.strip()
    return df


# Apply before uploading to Google Sheets
events_df = clean_dataframe_text(events_df)

# filter out events before today based on eventStartTime
# convert eventStartTime to datetime
events_df['eventStartTime'] = pd.to_datetime(events_df['eventStartTime'])

events_df = events_df[events_df['eventStartTime'] > pd.Timestamp.today()]
# filter
events_df

,Dupe,CONCAT,eventName,eventURL,Submitted,eventStartTime,eventendTime,eventVenueName,eventAddress,eventCity,eventState,groupName,eventGoogleMaps,event_description,datetimeScraped
87,1,AI in Higher Education Webinar with Sonja Schm...,AI in Higher Education Webinar with Sonja Schm...,https://dsi.umn.edu/events/ai-higher-education...,FALSE,2025-03-04 12:00:00,2025-03-04 13:00:00,Online,,,,University of Minnesota,,Topic: “The AI Monoculture: Preserving Human D...,2025-01-26 9:44:10
88,1,03/04/2025 – 03/18/2025: QuickBooks Online Bas...,03/04/2025 – 03/18/2025: QuickBooks Online Bas...,https://www.womenventure.org/event/03-04-2025-...,FALSE,2025-03-04 17:00:00,2025-03-18 19:00:00,"165 Western Avenue North, Suite 8, Office #100...",,,,WomenVenture,,"Free, QuickBooks Online training to support bu...",
90,1,Social and Q&A Meeting45720.7916666667,Social and Q&A Meeting,https://www.meetup.com/miniapples/events/30614...,FALSE,2025-03-04 19:00:00,2025-03-04 20:00:00,Online,Online,,,Mini’app’les - the Minnesota Apple Users' Grou...,,Come join us for a virtual social and Q&A meet...,2024-12-28 21:49:03.584855+00:00
92,1,Practical Cloud Cost Control: Insights and Str...,Practical Cloud Cost Control: Insights and Str...,https://www.meetup.com/platform-engineering-mi...,FALSE,2025-03-05 09:00:00,2025-03-05 10:00:00,Online,Online,,,Platform Engineering Minneapolis,,"Dear Platform Engineering Community,\n\nSuper ...",2025-01-26 15:46:31.167329+00:00
94,1,Webinar Wednesdays Live Demo and Q+A Session w...,Webinar Wednesdays Live Demo and Q+A Session w...,https://www.meetup.com/silicon-prairie-fundrai...,FALSE,2025-03-05 11:00:00,2025-03-05 11:30:00,Online,Online,,,Silicon Prairie Portal and Exchange Meetups!,,Thinking about raising capital but not sure wh...,2025-02-02 8:27:36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,1,Azure User Group: TBD45995.6666666667,Azure User Group: TBD,https://www.meetup.com/minneapolis-azure-cloud...,FALSE,2025-12-04 16:00:00,2025-12-04 17:15:00,Online,Online,,,Minnesota Azure User Group,,Please join us for the monthly Minnesota Azure...,
303,1,"Blockchain 101 : What is Bitcoin, ICO's and ot...","Blockchain 101 : What is Bitcoin, ICO's and ot...",https://www.meetup.com/blockchains-in-the-real...,FALSE,2025-12-04 18:00:00,2025-12-04 20:00:00,Online,Online,,,Blockchains in the Real World,,Blockchain Curious? Blockchain Professional?\n...,
304,1,Monthly Gathering 46007.6666666667,Monthly Gathering,https://www.meetup.com/twin-cities-agile-coach...,FALSE,2025-12-16 16:00:00,2025-12-16 17:00:00,Online,Online,,,Twin Cities Agile Coach Group,,"Agenda:\n4:00pm - 4:15pm CT - intros, networki...",
305,1,(ONLINE) Midwest Bootstrappers Breakfast 46009...,(ONLINE) Midwest Bootstrappers Breakfast,https://www.meetup.com/bootstrappers-breakfast...,FALSE,2025-12-18 08:30:00,2025-12-18 10:00:00,Online,Online,,,Bootstrappers Breakfast - Minneapolis Entrepre...,,Bootstrappers Breakfast is currently virtual i...,


## Submit Events to Minnestar's Calendar

In [5]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

url = "https://minnestar.org/submit-event/"
sleep_timer = 1  # how long to wait (in seconds) after each new web page submission, mostly to just be polite to the server load

# driver = webdriver.Firefox(options=options)

driver.get(f'{url}')
time.sleep(sleep_timer)

In [11]:
events_to_skip = ['Topic Meeting', 'Meetup Topic TBA', 'She Talks Data with guest speaker to be announced!',
                  'Azure User Group: TBD']


def clean_text_for_chrome(text):
    if not isinstance(text, str):
        return str(text)
    # Replace emojis and other non-BMP characters with a space
    return ''.join(char if ord(char) < 65536 else ' ' for char in text)


cnt = 0

# iterate through the events
for idx, row in events_df.iterrows():
    if row['eventName'] in events_to_skip:
        continue
    if row['Submitted'] == "TRUE":
        continue

    print(cnt, row['eventName'])
    driver.get(f'{url}')
    time.sleep(sleep_timer)
    event_title_xpath = '/html/body/main/section[2]/div[2]/section[1]/div/div/div/div[2]/div/form/div[2]/div/div[2]/div/input'
    event_location_xpath = '/html/body/main/section[2]/div[2]/section[1]/div/div/div/div[2]/div/form/div[2]/div/div[3]/div/input'
    event_start_time_xpath = '/html/body/main/section[2]/div[2]/section[1]/div/div/div/div[2]/div/form/div[2]/div/div[5]/div/input'
    event_end_time_xpath = '/html/body/main/section[2]/div[2]/section[1]/div/div/div/div[2]/div/form/div[2]/div/div[6]/div/input'
    event_description_xpath = '/html/body/main/section[2]/div[2]/section[1]/div/div/div/div[2]/div/form/div[2]/div/div[7]/div/textarea'
    submitted_by_xpath = '/html/body/main/section[2]/div[2]/section[1]/div/div/div/div[2]/div/form/div[2]/div/div[8]/div/input'
    email_xpath = '/html/body/main/section[2]/div[2]/section[1]/div/div/div/div[2]/div/form/div[2]/div/div[9]/div/input'
    more_info_url_xpath = '/html/body/main/section[2]/div[2]/section[1]/div/div/div/div[2]/div/form/div[2]/div/div[10]/div/input'
    rsvp_url_xpath = '/html/body/main/section[2]/div[2]/section[1]/div/div/div/div[2]/div/form/div[2]/div/div[12]/div/input'
    tags_xpath = '/html/body/main/section[2]/div[2]/section[1]/div/div/div/div[2]/div/form/div[2]/div/div[13]/div/input'
    submit_button_xpath = '/html/body/main/section[2]/div[2]/section[1]/div/div/div/div[2]/div/form/div[3]/input[1]'

    # insert the eventName into the form
    event_title = driver.find_element(By.XPATH, event_title_xpath)
    event_title.send_keys(clean_text_for_chrome(row['eventName']))

    # insert the eventLocation into the form
    event_location = driver.find_element(By.XPATH, event_location_xpath)
    event_location.send_keys(clean_text_for_chrome(row['eventVenueName']))

    # insert the eventStartTime into the form
    event_start_time = driver.find_element(By.XPATH, event_start_time_xpath)
    start_time_str = row['eventStartTime'].strftime('%m/%d/%Y %I:%M %p')
    event_start_time.send_keys(start_time_str)
    # print(row['eventStartTime'], start_time_str)

    # insert the eventEndTime into the form if it's not null
    event_end_time = driver.find_element(By.XPATH, event_end_time_xpath)
    # print(row['eventendTime'])

    end_time_str = row['eventendTime']

    end_time_str = end_time_str.replace('/', '-')

    if isinstance(end_time_str, pd.Timestamp):
        end_time_str = end_time_str.strftime('%m/%d/%Y %I:%M %p')

    elif len(end_time_str) > 0:
        try:
            end_time_str = datetime.strptime(end_time_str, '%Y-%m-%d %H:%M:%S')
        except Exception as e:
            try:
                end_time_str = datetime.strptime(end_time_str, '%Y-%m-%d %H:%M')
            except Exception as e:
                end_time_str = datetime.strptime(end_time_str, '%Y-%m-%d %I:%M %p')
        end_time_str = end_time_str.strftime('%m/%d/%Y %I:%M %p')
    else:
        end_time_str = str(end_time_str)
    event_end_time.send_keys(end_time_str)

    # insert the eventDescription into the form
    event_description = driver.find_element(By.XPATH, event_description_xpath)
    event_description.send_keys(clean_text_for_chrome(row['event_description']))

    # insert the submittedBy into the form
    submitted_by = driver.find_element(By.XPATH, submitted_by_xpath)
    submitted_by.send_keys('Bo Driscoll')

    # insert the email into the form
    email = driver.find_element(By.XPATH, email_xpath)
    email.send_keys('driscoll42@gmail.com')

    # insert the RSVP URL into the form
    rsvp_url = driver.find_element(By.XPATH, rsvp_url_xpath)
    rsvp_url.send_keys(row['eventURL'])

    # click the submit button
    submit_button = driver.find_element(By.XPATH, submit_button_xpath)
    submit_button.click()

    # update Submitted to True in events_df
    events_df.at[idx, 'Submitted'] = True

    time.sleep(5)

    cnt += 1



0 AI in Higher Education Webinar with Sonja Schmer-Galunder
1 03/04/2025 – 03/18/2025: QuickBooks Online Basic Training
2 Social and Q&A Meeting
3 Practical Cloud Cost Control: Insights and Strategies (Virtual MeetUp)
4 Webinar Wednesdays Live Demo and Q+A Session with Silicon Prairie!
5 Power BI Meetup | Bald Man Brewing
6 TechConnect - Virtual Job and Networking Event for Data AI Tech Professionals
7 MN Mingle @ Expo West 2025
8 Fireside Chat: Simple Abundance
9 NICAR conference
10 [AI Alliance] Introducing Gneissweb: A State-Of-The-Art LLM Pre-training Dataset
11 WomenVenture Overview: 3/6/25
12 The SEO Implementation Blueprint: Tools, Tactics & Techniques
13 CyberConnect - Virtual Career Networking for Cybersecurity Professionals
14 Lazy DevOps for .NET
15 TC Drupal Happy Hour
16 Blockchain 101 : What is Bitcoin, ICO's and other uses of this technology?
17 Silicon Prairie / Investment Crowdfunding / Ask Me Anything!
18 AnalyticsCLUB JobExpo Virtual Business, Data & Technology Job E